In [9]:
import sys
import os
import astropy.io.fits as fits
import math
import os.path
from numba import njit#
from astropy.visualization import astropy_mpl_style
from astropy.utils.data import get_pkg_data_filename
from astropy.io import fits
import matplotlib.pyplot as plt
import operator
from matplotlib.colors import LogNorm
from photutils.datasets import make_4gaussians_image
from photutils import centroid_com, centroid_1dg, centroid_2dg
import numpy as np
from astropy.visualization import PercentileInterval
from photutils import CircularAperture
from photutils import aperture_photometry
from astropy.io import ascii
from astropy.table import Table, Column, MaskedColumn

In [10]:
# definitions
def slicing_im(npix,box):
    npix = npix #total pixels
    box = box #changes the display of arrays
    result= (npix - box)/2
#     print(result)
#     print(result)
    if result % 1 == 0:
        end_num = int(result)
        strt_num = end_num
        new_st = strt_num
        new_end = npix - end_num
    else:
#         end_num = int(round(result,0))
        end_num = result +0.5
        strt_num = end_num - 1
        new_st = strt_num 
        new_end = npix - end_num
#     print("total pixels is: ",npix,'\nLeft side start by:',new_st,"\nRight side end by:",new_end)
    strt = int(new_st)
    end = int(new_end)
    return strt, end

def scale_image(scale,time, data_image):
    scale_num = scale
    interval = PercentileInterval(scale_num)
    interval.get_limits(data_image[time,:,:])
    scale_array = interval(data_image[time,:,:])
    return scale_array 

def diff_arr(array):
    new_array = []
    for i in range(len(array)):
        result = array[i]-i-1
        new_array.append(result)
    return new_array

def aperture_sum(radius, data, val_x, val_y):
    R = radius
    val_2dg_x = val_x
    val_2dg_y = val_y
    # for positions we will take the 2dg Gaussian outputs.
    positions = [(val_2dg_x, val_2dg_y)]
    apertures = CircularAperture(positions, r=R)
    # print(apertures)
    phot_table = aperture_photometry(data,apertures)
    return phot_table['aperture_sum']
# definitions
def slicing_im(npix,box):
    npix = npix #total pixels
    box = box #changes the display of arrays
    result= (npix - box)/2
#     print(result)
#     print(result)
    if result % 1 == 0:
        end_num = int(result)
        strt_num = end_num
        new_st = strt_num
        new_end = npix - end_num
    else:
#         end_num = int(round(result,0))
        end_num = result +0.5
        strt_num = end_num - 1
        new_st = strt_num 
        new_end = npix - end_num
#     print("total pixels is: ",npix,'\nLeft side start by:',new_st,"\nRight side end by:",new_end)
    strt = int(new_st)
    end = int(new_end)
    return strt, end

def scale_image(scale,time, data_image):
    scale_num = scale
    interval = PercentileInterval(scale_num)
    interval.get_limits(data_image[time,:,:])
    scale_array = interval(data_image[time,:,:])
    return scale_array

def org_dict(dict_name,str_num_name,dg_x,dg_y):
    start_ind =0 
    end_ind = 15
    str_arr_list = ["cen_2dg_arr_x","cen_2dg_arr_y"]
    dict_cen = dict_name
# puts each into a seperate array of 15 indexes
    for i in range(0,64):
#         dict_cen[str_arr_list[0]+'_'+str(i)] = com_x[start_ind:end_ind]
#         dict_cen[str_arr_list[1]+'_'+str(i)] = com_y[start_ind:end_ind]
        dict_cen[str_arr_list[0]+'_'+str(i)] = dg_x[start_ind:end_ind]
        dict_cen[str_arr_list[1]+'_'+str(i)] = dg_y[start_ind:end_ind]
        start_ind = start_ind + 15
        end_ind = end_ind+15
#creates an array of numbered names.
    str_num_arr=str_num_name
    for j in range(0,64):
        str_name_1 = str_arr_list[0]+'_'+str(j)
        str_name_2 = str_arr_list[1]+'_'+str(j)
        str_num_arr.append(str_name_1)
        str_num_arr.append(str_name_2)
        
def time_arr_func(time_arr,ind_time_arr,frame_t,bmjd_obs):
    frameday = frame_t/(3600*24)
    shift_bmjd = bmjd_obs + 0.5* frameday
    list_bmjd=[shift_bmjd]
    time_arr.append(shift_bmjd)
    ind_time_arr.append(shift_bmjd)
    # Adding onto array.
    for j in range(63):
        value = list_bmjd[0] + (j+1)*frameday
        ind_time_arr.append(value)
        time_arr.append(value)

In [11]:
created_dir ='/Users/melaniapena/Rsrch/Luhman_16_Research/ap_phot_ascii_2dg'
name_of_file_last = '/2dg_ap_ph_prt'
val_range = 8
new_dir_arr=[]
num = np.linspace(1,val_range-1,val_range-1)

for n_index in range(val_range):
#     num = [1,2,3,4,5,6,7]
    if n_index == 5:
        part_inp = str(6)
        part_num =name_of_file_last+part_inp+"-1"
    elif n_index == 6:
        part_inp = str(6)
        part_num =name_of_file_last+part_inp+"-2"
    elif n_index == 7:
        part_inp = str(7)
        part_num =name_of_file_last+part_inp
    else:
        part_inp = str(int(num[n_index]))
        part_num =name_of_file_last+part_inp
#     print(part_num)
#     des_var_dir = desired_var + part_num

    save_dir = created_dir + part_num+'/'
    new_dir_arr.append(save_dir) 

# #   Checks to see if the the folder exist and if it's not, it creates it. 
# #   Otherwise, it is not need since it is already created.
#     print("Created directory:",save_dir)
#     if not os.path.exists(save_dir):
#         os.makedirs(save_dir)
#         new_dir_arr.append(save_dir)
new_dir_arr

['/Users/melaniapena/Rsrch/Luhman_16_Research/ap_phot_ascii_2dg/2dg_ap_ph_prt1/',
 '/Users/melaniapena/Rsrch/Luhman_16_Research/ap_phot_ascii_2dg/2dg_ap_ph_prt2/',
 '/Users/melaniapena/Rsrch/Luhman_16_Research/ap_phot_ascii_2dg/2dg_ap_ph_prt3/',
 '/Users/melaniapena/Rsrch/Luhman_16_Research/ap_phot_ascii_2dg/2dg_ap_ph_prt4/',
 '/Users/melaniapena/Rsrch/Luhman_16_Research/ap_phot_ascii_2dg/2dg_ap_ph_prt5/',
 '/Users/melaniapena/Rsrch/Luhman_16_Research/ap_phot_ascii_2dg/2dg_ap_ph_prt6-1/',
 '/Users/melaniapena/Rsrch/Luhman_16_Research/ap_phot_ascii_2dg/2dg_ap_ph_prt6-2/',
 '/Users/melaniapena/Rsrch/Luhman_16_Research/ap_phot_ascii_2dg/2dg_ap_ph_prt7/']

In [12]:
tot_dir_bcd = ['/Users/melaniapena/Rsrch/Luhman_16_Research/bcd_files/only_bcd_part_1',
           '/Users/melaniapena/Rsrch/Luhman_16_Research/bcd_files/only_bcd_part_2',
           '/Users/melaniapena/Rsrch/Luhman_16_Research/bcd_files/only_bcd_part_3',
           '/Users/melaniapena/Rsrch/Luhman_16_Research/bcd_files/only_bcd_part_4',
           '/Users/melaniapena/Rsrch/Luhman_16_Research/bcd_files/only_bcd_part_5',
           '/Users/melaniapena/Rsrch/Luhman_16_Research/bcd_files/only_bcd_part_6-1',
           '/Users/melaniapena/Rsrch/Luhman_16_Research/bcd_files/only_bcd_part_6-2',
           '/Users/melaniapena/Rsrch/Luhman_16_Research/bcd_files/only_bcd_part_7',]

# str_arr_list = ["cen_com_arr_x", "cen_com_arr_y","cen_2dg_arr_x","cen_2dg_arr_y"]
col_1='AperturePhot'
col_2='bmjd'

part_num = 2
ori_dir = new_dir_arr[part_num-1]

print(ori_dir)


/Users/melaniapena/Rsrch/Luhman_16_Research/ap_phot_ascii_2dg/2dg_ap_ph_prt2/


In [14]:
phot_ap_arr =[]
time_arr = []
cen_2dg_arr_x=[]
cen_2dg_arr_y=[]

m = 0

scale_num = 100
npix = 32
start_i = 3
steps_i = 2

start_ind =0 
end_ind = 15

if part_num == 2 or part_num == 4 or part_num == 5:
    r_1 = 6
    r_2 = 12
    r_3 = 14
else:
    r_1 = 8
    r_2 = 13
    r_3 = 23
r_1

6

In [ ]:
# only focus on COM centroid for aperture photometry
for filename in os.listdir(tot_dir_bcd[part_num-1]):
    if filename.endswith(".fits"): 
#         dir_bcd ='/Users/melaniapena/Rsrch/Luhman_16_Research/bcd_files/only_bcd_part_1/SPITZER_I1_50037760_0000_0000_2_bcd.fits'
        dir_bcd = os.path.join(tot_dir_bcd[part_num-1],filename)
        fits_file = fits.open(dir_bcd)
        fits_data = get_pkg_data_filename(dir_bcd)
        list_Primary = fits_file[0]
        image_data= fits.getdata(fits_data, ext=0)
        
        frame_t = fits_file[0].header['FRAMTIME']
        bmjd_obs = fits_file[0].header['BMJD_OBS']
#         print('bmjd observation:', bmjd_obs)
        flux_conv= fits_file[0].header['FLUXCONV']
#         print('Flux Conversion',flux_conv)
        gain = fits_file[0].header['GAIN']
#         print('Gain',gain)
        exp_time= fits_file[0].header['EXPTIME']
#         print('Exp-time', exp_time)
        conversion_factor = gain*exp_time/flux_conv
#         print(conversion_factor,'\n')
        counts_image = image_data * conversion_factor
        num = 0
        centroid_2dg_arr_x = []
        centroid_2dg_arr_y = []
        centroid_2dg_arr_xtot=[]
        centroid_2dg_arr_ytot=[]
        for time_i in range(0,64): 
            for i_box in range(start_i,npix,steps_i):
                strt, end = slicing_im(npix,i_box)
                interval = PercentileInterval(scale_num)
                interval.get_limits(counts_image[time_i,strt:end,strt:end])
                scale_array = interval(counts_image[time_i,strt:end,strt:end])
                sliced_image = counts_image[0,strt:end,strt:end] 
                result = scale_array
                if i_box < 3:
                    print('too small of an image to calculate\n')
                elif i_box >= 3:
                    x1, y1 = centroid_2dg(result)
#                     print(i_box,strt,end)
#                     print('Box Size: ', i_box,'x',i_box)
#                     print('2dg: ',x1,y1)
                    centroid_2dg_arr_x.append(x1)
                    centroid_2dg_arr_y.append(y1)
                    num = num+1
#                     print(num,time_i,'\n')
#                     plt.imshow(result)
#                     plt.show()
                    if i_box == 31:
                        centroid_2dg_arr_xtot.append(x1)
                        centroid_2dg_arr_ytot.append(y1)
                        cen_2dg_arr_x.append(x1)
                        cen_2dg_arr_y.append(y1)

#       the empty array and dictionary will help organize the list of centroids thats
#       taken out of each file.
        str_num_arr=[]      
        dict_cen = {}
#         The line below is a define function (found in previous cell)
#         that organizes the centroids.
        org_dict(dict_cen,str_num_arr,centroid_2dg_arr_x,centroid_2dg_arr_y)
              
#         -----------------------------------------------
#         Aperture Photometry

#         focuses on the indiviual array of the photo data.
        ind_photometry_arr =[]
        h_2dg = 0 ##starting index for 2dG
#         print(org_dict)
        for l in range(0,64):
            aperture_1 = aperture_sum(r_1,counts_image[l,:,:], dict_cen[str_num_arr[h_2dg]][14], dict_cen[str_num_arr[h_2dg+1]][14]) 
            aperture_2 = aperture_sum(r_2,counts_image[l,:,:], dict_cen[str_num_arr[h_2dg]][14], dict_cen[str_num_arr[h_2dg+1]][14])
            aperture_3 = aperture_sum(r_3,counts_image[l,:,:], dict_cen[str_num_arr[h_2dg]][14], dict_cen[str_num_arr[h_2dg+1]][14])
## Area of aperture,
            area_1 = math.pi * r_1**2
## Background in annulus from r_2 to r_3
            background_counts = aperture_3 - aperture_2
## Area of Annulus
            area_2 = math.pi*(r_3**2 - r_2**2) #also known as npix
## subtracting background: 
            backgrnd_subtract = aperture_1 - background_counts*area_1/area_2
            ind_photometry_arr.append(backgrnd_subtract)
            phot_ap_arr.append(backgrnd_subtract)
            h_2dg = h_2dg+2

#         -----------------------------------------------
#         Time array

        ind_time_arr = []
        time_arr_func(time_arr,ind_time_arr,frame_t,bmjd_obs)

#         -----------------------------------------------
#         Saves it into an ascii array

#         print(num)
#         plt.plot(ind_time_arr,photometry_arr)
#         plt.plot(photometry_arr)

        if part_num==6:
            str_m='_part_6-1'
        elif part_num==7:
            str_m='_part_6-2'
        elif part_num==8:
            str_m='_part_7'
        else:
            str_m='_part_'+str(part_num)
        
        word=ori_dir+'bcdfits_'+str(m)+str_m+'.txt'
        print(m)
        m+=1
#         plt.savefig(word)
        ind_arr_table = Table([ind_photometry_arr,ind_time_arr,centroid_2dg_arr_xtot,centroid_2dg_arr_ytot], names=[col_1,col_2,'x_2dG_cen','y_2dG_cen'])
        ascii.write(ind_arr_table, word, format='tab', fast_writer=False) 
#         plt.show()    
#         -----------------------------------------------

    else:
        continue

if part_num==6:
    str_m='part_6-1'
elif part_num==7:
    str_m='part_6-2'
elif part_num==8:
    str_m='part_7'
else:
    str_m='part_'+str(part_num)


word=ori_dir+'bcdfits_'+str_m+'.txt'
Tot_arr_table = Table([phot_ap_arr,time_arr,cen_2dg_arr_x,cen_2dg_arr_y], names=[col_1,col_2,'x_2dg_cen','y_2dg_cen'])
ascii.write(Tot_arr_table, word, format='tab', fast_writer=False) 
        

/anaconda3/envs/astroconda/lib/python3.5/site-packages/skimage/measure/_moments.py:258: UserWarning: Using deprecated 2D-only, xy-coordinate interface to moments_central. This interface will be removed in scikit-image 0.16. Use moments_central(image, center=(cr, cc), order=3).
  warn(message)


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
